In [1]:
import numpy as np
import pandas as pd

from dask.distributed import Client

from condor_utils import make_htcondor_cluster

In [2]:
cluster = make_htcondor_cluster(local=False, dashboard_address=13346)
cluster

In [3]:
cluster.scale(5)

In [6]:
client = Client(cluster)
client

Client Scheduler: tcp://169.228.130.37:6822 Dashboard: http://169.228.130.37:13346/status,Cluster Workers: 1 Cores: 1 Memory: 2.00 GB


In [5]:
def f():
    import subprocess

    script="""
source /cvmfs/cms.cern.ch/cmsset_default.sh 
cd /cvmfs/cms.cern.ch/slc7_amd64_gcc700/cms/cmssw/CMSSW_10_5_0/ 
eval `scramv1 runtime -sh` 
cd - 
root --help
python /hadoop/cms/store/user/hmei/testRoot.py

cp /hadoop/cms/store/user/hmei/combineStandalone.tar.gz .
tar -xzf combineStandalone.tar.gz
pwd

# https://root-forum.cern.ch/t/error-in-cling-insertintoautoloadingstate/29347
# Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState
export ROOT_INCLUDE_PATH=/srv/temp/

cd HiggsAnalysis/CombinedLimit/
source ./env_standalone.sh 
combine -h

#which combine
"""
    with open("temp.sh","w") as fh:
        fh.write(script)
    _ = subprocess.getoutput("chmod u+x temp.sh")
    cmd = "time ./temp.sh"
    return subprocess.getoutput(cmd)
print(client.submit(f).result())

/srv/temp

Usage: root [-l] [-b] [-n] [-q] [dir] [[file:]data.root] [file1.C ... fileN.C]
Options:
  -b : run in batch mode without graphics
  -x : exit on exception
  -e expression: request execution of the given C++ expression
  -n : do not execute logon and logoff macros as specified in .rootrc
  -q : exit after processing command line macro files
  -l : do not show splash screen
  -t : enable thread-safety and implicit multi-threading (IMT)
 --notebook : execute ROOT notebook
 dir : if dir is a valid directory cd to it before executing

  -?      : print usage
  -h      : print usage
  --help  : print usage
  -config : print ./configure options
  -memstat : run with memory usage monitoring

3
/srv/temp
Usage: combine datacard [options]
Main options:
  -d [ --datacard ] arg                 Datacard file (can also be specified 
                                        directly without the -d or --datacard)
  -M [ --method ] arg (=AsymptoticLimits)
                                     

In [7]:
import time
import random
futures = client.map(lambda x:time.sleep(0.2*random.random()), range(50))
results = client.gather(futures)

In [ ]:
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client